In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/mtfishman/ITensorQTT.jl.git

Cloning into 'ITensorQTT.jl'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 482 (delta 9), reused 10 (delta 4), pack-reused 458
Receiving objects: 100% (482/482), 722.10 KiB | 11.28 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [ ]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.9/julia-1.9.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

Now switch runtime to Julia

In [1]:
using Pkg
Pkg.add("ITensors")
Pkg.add("ITensorTDVP")
Pkg.add("FFTW")
Pkg.add("Polynomials")
Pkg.add("KrylovKit")
using ITensors
using FFTW

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
include("./ITensorQTT.jl/src/ITensorQTT.jl")
using .ITensorQTT

In [3]:
Pkg.add("TimerOutputs")
using TimerOutputs

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [4]:
Pkg.add("PyCall")
using PyCall

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [5]:
using DelimitedFiles

In [6]:
ITensors.set_warn_order(30)

14

In [7]:
# TimingDriver: n_qubits = number of qubits
# func = function name
#    -options = [gaussian, cos, cusps, fft]
# _cutoff = cutoff during mps and mpo construction
# _to = TimerOutput that saves the runtimes

function TimingDriver(n_qubits::Int64, _func::String, _cutoff::Float64, _to)
  s = siteinds("Qubit", n_qubits)
  if _func == "gaussian"
    x_not = 0.5
    sigma = 0.1
    f1(x) = exp(-(x - x_not)^2 / sigma^2)
    psi = @timeit _to "gaus: function_to_mps: $n_qubits" function_to_mps(f1, s, 0.0, 1.0; cutoff=_cutoff)
    f_psi = @timeit _to "gaus: apply_dft_mpo: $n_qubits" apply_dft_mpo(psi; cutoff=_cutoff)
  elseif _func == "cos"
    f2(x) = cos(2*pi*x)
    psi = @timeit _to "cos: function_to_mps: $n_qubits" function_to_mps(f2, s, 0.0, 1.0; cutoff=_cutoff)
    f_psi = @timeit _to "cos: apply_dft_mpo: $n_qubits" apply_dft_mpo(psi; cutoff=_cutoff)
  elseif _func == "cusps"
    f3(x) = cos(2*π*x) + 2*exp(-3*abs(x-0.2))
    psi = @timeit _to "cusps: function_to_mps: $n_qubits" function_to_mps(f3, s, 0.0, 1.0; cutoff=_cutoff)
    f_psi = @timeit _to "cusps: apply_dft_mpo: $n_qubits" apply_dft_mpo(psi; cutoff=_cutoff)
  elseif _func == "fft"
    @timeit _to "fft: $n_qubits" fft([x for x=0:(2^n_qubits)])
  end
  return
end

TimingDriver (generic function with 1 method)

In [ ]:
# Initialize Timer
to = TimerOutput()
start = 4
stop = 10

# Run TimingDriver for functions from qubits=start to qubits=stop
for i in start:stop
  TimingDriver(i, "gaussian", 1e-15, to)
  TimingDriver(i, "cos", 1e-15, to)
  TimingDriver(i, "cusps", 1e-15, to)
  TimingDriver(i, "fft", 1e-15, to)
end
show(to)

In [ ]:
# Here, we take the runtimes for each function and operation and put them into an array

#=================GAUS================#
gaus_ftm = [TimerOutputs.time(to["gaus: function_to_mps: $x"]) for x=start:stop]
gaus_adm = [TimerOutputs.time(to["gaus: apply_dft_mpo: $x"]) for x=start:stop]

#=================COS=================#
cos_ftm = [TimerOutputs.time(to["cos: function_to_mps: $x"]) for x=start:stop]
cos_adm = [TimerOutputs.time(to["cos: apply_dft_mpo: $x"]) for x=start:stop]

#================CUSPS================#
cusps_ftm = [TimerOutputs.time(to["cusps: function_to_mps: $x"]) for x=start:stop]
cusps_adm = [TimerOutputs.time(to["cusps: apply_dft_mpo: $x"]) for x=start:stop]

#=================FFT=================#
fft_time = [TimerOutputs.time(to["fft: $x"]) for x=start:stop]

In [ ]:
# We start by deleting the existing csv containing the times for each function and operation
# We then write to a new csv for each function and operation

#============GAUS================#
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_function_to_mps.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_function_to_mps.csv",  gaus_ftm, ',')

run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_apply_dft_mpo.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_apply_dft_mpo.csv",  gaus_adm, ',')

#============COS=================#
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/cos_function_to_mps.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/cos_function_to_mps.csv",  cos_ftm, ',')

run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/cos_apply_dft_mpo.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/cos_apply_dft_mpo.csv",  cos_adm, ',')

#================CUSPS================#
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_function_to_mps.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_function_to_mps.csv",  cusps_ftm, ',')

run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_apply_dft_mpo.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_apply_dft_mpo.csv",  cusps_adm, ',')

#================FFT=================#
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/fft.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/fft.csv",  fft_time, ',')


In [ ]:
# Python function that reads from the csv's and saves a graph from matplotlib

py"""
def graph():
  import matplotlib.pyplot as plt
  import pandas
  plt.clf()

  #=======GAUSSIAN=========
  gaus_adm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_apply_dft_mpo.csv', header=None)
  gaus_ftm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_function_to_mps.csv', header=None)

  plt.plot(range(4,gaus_adm.shape[0]+4), gaus_adm[0], linestyle='-', marker='^', color='b', label="Gaus: DFT MPO")
  plt.plot(range(4,gaus_adm.shape[0]+4), gaus_adm[0]+gaus_ftm[0], linestyle='--', marker='^', color='b', label="Gaus: DFT MPO + MPS")

  #===========COSINE=================
  cos_adm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/cos_apply_dft_mpo.csv', header=None)
  cos_ftm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/cos_function_to_mps.csv', header=None)

  plt.plot(range(4,cos_adm.shape[0]+4), cos_adm[0], linestyle='-', marker='o', color='r', label="Cos: DFT MPO")
  plt.plot(range(4,cos_adm.shape[0]+4), cos_adm[0]+cos_ftm[0], linestyle='--', marker='o', color='r', label="Cos: DFT MPO + MPS")

  #================CUSPS===================
  cusps_adm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_apply_dft_mpo.csv', header=None)
  cusps_ftm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_function_to_mps.csv', header=None)

  plt.plot(range(4,cusps_adm.shape[0]+4), cusps_adm[0], linestyle='-', marker='s', color='g', label="Cusps: DFT MPO")
  plt.plot(range(4,cusps_adm.shape[0]+4), cusps_adm[0]+cusps_ftm[0], linestyle='--', marker='s', color='g', label="Cusps: DFT MPO + MPS")

  #====================FFT=======================
  fft = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/fft.csv', header=None)

  plt.plot(range(4,fft.shape[0]+4), fft[0], linestyle='-', marker='*', color='k', label="FFT")

  plt.title("Julia")
  plt.yscale(value='log')
  plt.xlabel("Number of qubits")
  plt.ylabel("Time takes (ns)")
  plt.legend()
  plt.savefig('drive/MyDrive/Colab Notebooks/github/julia_qft/graph.png')
"""

In [ ]:
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/graph.png`)
py"graph"()

In [ ]:
#run(`rm -f graph.png`)
#run(`python temp.py`)

Process(`python temp.py`, ProcessExited(0))

In [ ]:
n = 26
@time fft([x for x=0:(2^n)])

 17.935111 seconds (13 allocations: 2.500 GiB, 0.83% gc time)


67108865-element Vector{ComplexF64}:
 2.2517998472396795e15 + 0.02685546875im
 -3.3554432537658736e7 + 7.167701637642468e14im
 -3.3554432522277914e7 + 3.583850818821225e14im
  -3.355443245285036e7 + 2.389233879214142e14im
  -3.355443249160769e7 + 1.791925409410597e14im
 -3.3554432484298736e7 + 1.433540327528468e14im
   -3.35544324921723e7 + 1.1946169396070472e14im
 -3.3554432552146934e7 + 1.023957376806031e14im
  -3.355443248931886e7 + 8.959627047052672e13im
  -3.355443248384857e7 + 7.964112930713388e13im
  -3.355443247052767e7 + 7.16770163764195e13im
   -3.35544324973602e7 + 6.51609239785622e13im
  -3.355443250921634e7 + 5.973084698034766e13im
                       ⋮
   -3.35544325066384e7 - 5.973084698034766e13im
 -3.3554432472187884e7 - 6.516092397856217e13im
  -3.355443248314723e7 - 7.167701637641947e13im
   -3.35544324365563e7 - 7.964112930713388e13im
 -3.3554432486310095e7 - 8.959627047052673e13im
  -3.355443246453246e7 - 1.0239573768060306e14im
  -3.355443251356994e7 - 1.194616